In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
sys.path.insert(0, os.path.abspath('..'))
from hsmm4acc import hsmm
from milleniumcohort import create_config

In [3]:
config = create_config('../config.yml')
config.create_data_paths()

## Load model

In [4]:
import pickle
with open(config.model_file, 'r') as f:
    model = pickle.load(file=f)

In [16]:
o = model.init_state_distn
o.params

{'weights': array([  1.42868119e-02,   1.64341976e-01,   7.48422563e-01,
          1.58342192e-03,   2.81906414e-03,   5.25747147e-04,
          2.98556012e-03,   4.38375754e-02,   8.97896356e-04,
          2.02993837e-02])}

In [5]:
for o in model.obs_distns:
    print(o.params)

{'mu': array([ 0.02097878]), 'sigma': array([[ 0.00025252]])}
{'mu': array([  3.99831764e-05]), 'sigma': array([[  4.95653866e-07]])}
{'mu': array([ 0.00580844]), 'sigma': array([[  2.28461432e-05]])}
{'mu': array([ 0.06318959]), 'sigma': array([[ 0.00152634]])}
{'mu': array([ 0.10439295]), 'sigma': array([[ 0.00454811]])}
{'mu': array([ 0.19950354]), 'sigma': array([[ 0.01351362]])}
{'mu': array([ 0.54768306]), 'sigma': array([[ 0.27342921]])}
{'mu': array([ 0.07137205]), 'sigma': array([[ 0.00326894]])}
{'mu': array([ 0.08882429]), 'sigma': array([[ 0.00248588]])}
{'mu': array([ 0.02914308]), 'sigma': array([[ 0.00058561]])}


In [6]:
means = pd.DataFrame([o.params['mu'] for o in model.obs_distns], columns=config.hsmmconfig.column_names)
means.transpose().style.background_gradient(cmap='coolwarm',axis=1)

In [7]:
sigmas = pd.DataFrame([o.params['sigma'][0] for o in model.obs_distns], columns=config.hsmmconfig.column_names)
sigmas.transpose().style.background_gradient(cmap='coolwarm',axis=1)

In [8]:
durations = pd.DataFrame([o.params['lmbda'] for o in model.dur_distns], columns=['duration'])
durations['duration_sec'] = ["{} sec".format(int(round(x*5))) for x in durations['duration']]
durations.transpose()

,0,1,2,3,4,5,6,7,8,9
duration,103.987,111.407,27.7678,141.885,35.6503,34.5231,3.09051,2.19207,91.6481,20.7052
duration_sec,520 sec,557 sec,139 sec,709 sec,178 sec,173 sec,15 sec,11 sec,458 sec,104 sec


In [9]:
# Transition probablilites
pd.DataFrame(model.trans_distn.trans_matrix).style.background_gradient(cmap='BuGn')

In [10]:
concat = pd.concat([means, sigmas], axis=1, keys=['mean', 'sigma'])
concat = concat.reorder_levels(order=[1,0], axis=1)
concat[('duration', 'lmbda')] = durations['duration_sec']

In [11]:
concat[concat.columns.sort_values()]

acceleration               duration
          mean         sigma    lmbda
0     0.020979  2.525220e-04  520 sec
1     0.000040  4.956539e-07  557 sec
2     0.005808  2.284614e-05  139 sec
3     0.063190  1.526341e-03  709 sec
4     0.104393  4.548113e-03  178 sec
5     0.199504  1.351362e-02  173 sec
6     0.547683  2.734292e-01   15 sec
7     0.071372  3.268943e-03   11 sec
8     0.088824  2.485879e-03  458 sec
9     0.029143  5.856092e-04  104 sec

In [12]:
concat.sort_values([('acceleration', 'mean')])

acceleration               duration
          mean         sigma    lmbda
1     0.000040  4.956539e-07  557 sec
2     0.005808  2.284614e-05  139 sec
0     0.020979  2.525220e-04  520 sec
9     0.029143  5.856092e-04  104 sec
3     0.063190  1.526341e-03  709 sec
7     0.071372  3.268943e-03   11 sec
8     0.088824  2.485879e-03  458 sec
4     0.104393  4.548113e-03  178 sec
5     0.199504  1.351362e-02  173 sec
6     0.547683  2.734292e-01   15 sec

In [12]:
concat.transpose().to_clipboard()